In [1]:
from DataGenerator import TimeSeriesGenerator
from conditions import scenarios
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from grouped_series import ExperimentTracker
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression

In [2]:
onehot1 = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

onehot2 = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

hybrid_vars = [0,1, 2, 3, 4, 5, 6, 7, -3, -2, -1]
hybrid_model = TimeSeriesGradientBoosting(model1 = LinearRegression(), model2 = XGBRegressor(), model1_variables = hybrid_vars)

hybrid_xgb_pipe = Pipeline([
    ('preprocessor', onehot1),
    ('regressor', hybrid_model)
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot2),
    ('regressor', SGDRegressor())
])

In [3]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithms=[hybrid_xgb_pipe, sgd_pipe], algorithm_name=["Gradient boosted decision tree", "Linear regression"], online=[False, False])
    return thesis


In [4]:
thesis = full_run()

Finished experiment! Elapsed time: 0.7166211605072021, total Elapsed time: 0.7176151275634766, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.6088011264801025, total Elapsed time: 1.3274121284484863, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.5100016593933105, total Elapsed time: 1.8374137878417969, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.513495922088623, total Elapsed time: 2.35090970993042, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished ex

In [5]:
results = thesis.resultsToDF()

In [6]:
results[["Algorithm","SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Algorithm","Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                                      mean   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             5.678333   
                                                                Small             3.496667   
                                                 Half observed  Large            14.508333   
                                                                Small             5.795000   
                                                 Unobserved     Large            12.180833   
                                                                Small             5.328333   
                               No Drift          No Drift       No Drift          3.380000   
                               Sudden Drift      Fully observed Large             5.382500   
                                                                Small             3.120833   
                                                 Half observed  Large             5.450000   
                                                                Small             4.610833   
                                                 Unobserved     Large            16.244167   
                                                                Small             6.156667   
                               Sudden Shock      3 days         Large             3.547500   
                                                                Small             3.442500   
                                                 7 days         Large             3.557500   
                                                                Small             3.415000   
Linear regression              Incremental Drift Fully observed Large             4.315833   
                                                                Small             3.384167   
                                                 Half observed  Large             7.833333   
                                                                Small             3.986667   
                                                 Unobserved     Large             7.547500   
                                                                Small             3.811667   
                               No Drift          No Drift       No Drift          3.202500   
                               Sudden Drift      Fully observed Large             4.396667   
                                                                Small             3.404167   
                                                 Half observed  Large             5.308333   
                                                                Small             3.439167   
                                                 Unobserved     Large             8.095833   
                                                                Small             3.940833   
                               Sudden Shock      3 days         Large             3.417500   
                                                                Small             3.232500   
                                                 7 days         Large             3.447500   
                                                                Small             3.247500   

                                                                                            \
                                                                                       std   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             2.793251   
                                                                Small             0.350229   
                                                 Half observed  Large            10.518592   
                                          

In [7]:
results.sort_values("SMAPE", ascending=False)   

,Algorithm,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
52,Gradient boosted decision tree,0.0,Sudden Drift,Large,Important,Unobserved,2084.583217,32.31
11,Gradient boosted decision tree,NaN,Sudden Drift,Large,Important,Unobserved,2050.824207,32.15
134,Gradient boosted decision tree,6.0,Sudden Drift,Large,Important,Unobserved,1961.504375,31.16
93,Gradient boosted decision tree,3.0,Sudden Drift,Large,Important,Unobserved,1919.339326,30.74
216,Linear regression,0.0,Sudden Drift,Large,Important,Unobserved,1894.468609,29.64
...,...,...,...,...,...,...,...,...
314,Linear regression,6.0,Incremental Drift,Large,Important,Fully observed,30.650648,2.82
255,Linear regression,3.0,Sudden Drift,Large,Important,Fully observed,30.319409,2.82
296,Linear regression,6.0,Sudden Drift,Large,Important,Fully observed,30.282155,2.81
3,Gradient boosted decision tree,NaN,Sudden Drift,Small,Medium,Fully observed,16.530015,2.78


Next up is changepoint/drift detection!
Let's start with doing it on a single series and then ramp up complexity to multivariate cases

In [8]:
from kats.detectors.bocpd import BOCPDetector, BOCPDModelType, TrendChangeParameters
from kats.consts import TimeSeriesData

C:\Users\chidi\anaconda3\envs\dbconnect\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
test_prod = TimeSeriesGenerator(size=365, amountOfVariables=10).toDataFrame()


In [10]:
# Add a time column to test_prod
test_prod["time"] = test_prod.index
test_prod = TimeSeriesData(test_prod)


In [11]:
det = BOCPDetector(test_prod)
changepoints = det.detector(model=BOCPDModelType.NORMAL_KNOWN_MODEL)

C:\Users\chidi\anaconda3\envs\dbconnect\lib\site-packages\ax\core\parameter.py:467: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "known_prec_multiplier". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

C:\Users\chidi\anaconda3\envs\dbconnect\lib\site-packages\ax\core\parameter.py:467: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "cp_prior". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.



TypeError: ufunc 'add' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''